In [1]:
# need to do this before transformer imports
import os
os.environ['HF_HOME'] = '/workspace/cache/huggingface/'

import os
os.chdir('/workspace/FutureGPT2/src/')
from evals.utils import *
from models.bigram_model import *
from models.mlp_model import *
from models.future_model import *
from data.utils import get_tokenizer
import datasets
from torch.utils.data import DataLoader
from torch import nn
from itertools import islice
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from torch import nn

from tqdm import tqdm
import pandas as pd
import gc
from glob import glob
import numpy as np
import copy
from num2words import num2words
from collections import defaultdict

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [2]:
model_name = 'mistralai/Mistral-7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
Token = {v: k for k, v in tokenizer.get_vocab().items()}

In [3]:
def print_tokens(s):
    tokens = tokenizer(s)['input_ids']
    print('|'.join(Token[t] for t in tokens))

In [4]:
def topk(v, k=10):
    # Takes in logits
    if isinstance(v, torch.Tensor):
        v = v.detach().cpu().numpy()
    v = v.flatten()
    idxs = v.argsort()[-k:][::-1]
    ret = [(Token[i], v[i]) for i in idxs]
    return pd.DataFrame(ret, columns=['token', 'logit'])

In [5]:
ckpt = glob(
    '/workspace/checkpoints/MISTRAL-NECK-SWEEP_*_hidden_idxs-31_hidden_lb-0_token_lb-0_neck_cls-lstm_*',
)[0]

In [6]:
ckpt

'/workspace/checkpoints/MISTRAL-NECK-SWEEP_20240102-191556-Ec6c4_hidden_idxs-31_hidden_lb-0_token_lb-0_neck_cls-lstm_epoch=00-val_self_loss=3.81.ckpt'

In [7]:
model = LitFutureModelWithNeck.load_from_checkpoint(ckpt, strict=False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
torch.set_grad_enabled(False)

In [9]:
input = tokenizer('Alice has two apples and Bob has four apples. In total, they have', return_tensors='pt').to('cuda')

In [10]:
nums = {i: num2words(i) for i in range(100) if len(tokenizer(num2words(i)).input_ids) == 2} # 1-token numbers

In [11]:
gc.collect(); torch.cuda.empty_cache()

In [12]:
from itertools import combinations_with_replacement as car
prompt = 'Alice has five apples and Bob has twelve apples. Combined, Alice and Bob have seventeen apples. Carol has {} apples and Dave has {} apples. Combined, Carol and Dave have' 

total = 0
base_topk = defaultdict(lambda: 0)
future_topk = defaultdict(lambda: 0)
ks = [1, 2, 3, 5, 10]
for i, j in tqdm(list(car(nums, 2))):
    total += 1
    gc.collect(); torch.cuda.empty_cache()
    input = tokenizer(
        prompt.format(nums[i], nums[j]),
        return_tensors='pt'
    ).to('cuda')
    out = model(input)
    if i + j not in nums:
        continue
    base_tok = topk(out.logits[0,-1,:]).token.tolist()
    future_tok = topk(out.future_logits[0,-2,:]).token.tolist()
    for k in ks:
        if '▁' + nums[i+j] in base_tok[:k]:
            base_topk[k] += 1
        if '▁' + nums[i+j] in future_tok[:k]:
            future_topk[k] += 1
for k in base_topk:
    base_topk[k] /= total
for k in future_topk:
    future_topk[k] /= total

100%|██████████| 300/300 [01:08<00:00,  4.41it/s]


In [13]:
base_topk

defaultdict(<function __main__.<lambda>()>,
            {1: 0.37,
             2: 0.38,
             3: 0.38666666666666666,
             5: 0.3933333333333333,
             10: 0.3933333333333333})

In [14]:
future_topk

defaultdict(<function __main__.<lambda>()>,
            {10: 0.06666666666666667,
             1: 0.006666666666666667,
             2: 0.013333333333333334,
             3: 0.016666666666666666,
             5: 0.023333333333333334})

In [25]:
input = tokenizer(
    #prompt.format('five', 'eleven'),
    #'Fried foods are high in cholesterol',
    'The director of the Manhattan project was J Robert Oppenheimer',
    return_tensors='pt'
).to('cuda')

In [26]:
out = model(input)

In [27]:
topk(out.logits[0, -2,:])

,token,logit
0,heimer,19.516129
1,h,13.359095
2,he,12.359792
3,heim,12.232853
4,her,10.319977
5,ha,10.146824
6,hem,10.030427
7,ham,9.772432
8,him,9.236302
9,-,8.828691


In [28]:
topk(out.future_logits[0,-3,:])

,token,logit
0,.,8.544149
1,",",8.523829
2,hen,8.339357
3,en,7.998469
4,ard,7.784670
5,man,7.768543
6,hal,7.739163
7,berg,7.691470
8,h,7.520702
9,pe,7.334779


In [19]:
out.future_logits.shape

torch.Size([1, 10, 32000])

In [20]:
'▁six' in topk(out.logits[0, -1,:]).token.tolist()

False